# Programming Assignment 3 - Simple Linear versus Ridge Regression (70 points)

Your friend Bob just moved to Boston. He is a real estate agent who is trying to evaluate the prices of houses in the Boston area. He has been using a linear regression model but it has been taking a long time to compute prices for new hourse. He comes to you for help as he knows that you're an expert in machine learning. As a pro, you suggest ridge regression to reduce the feature weights as a way to increase the speed of computation. Bob, however, being a skeptic isn't convinced. He wants you to write a program that illustrates the difference in training and test costs for both linear and ridge regression on the same dataset. Being a good friend, you oblige and hence this assignment :) 

In [1]:
import numpy as np
import sklearn
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import sklearn.linear_model

In [2]:
# TODO - Import the boston dataset from sklearn - 5 points. 
boston_data = sklearn.datasets.load_boston()

In [3]:
# TODO - Create X and Y variables - X holding the .data and Y holding .target - 5 points
X = boston_data.data
Y = boston_data.target

# TODO - Reshape Y to be a rank 2 matrix - 5 points
Y = Y.reshape(Y.shape[0],1)

In [4]:
# Add scaling and centering of data
X = sklearn.preprocessing.scale(X)

We will also create polynomial feeatures for the dataset to test linear and ridge regression on data with d = 1 and data with d = 2. Feel free to increase the # of degress and see what effect it has on the training and test error. 

In [5]:
# TODO - Create a PolynomialFeatures object with degree = 2. 
# Transform X and save it into X_2. Simply copy Y into Y_2 - 10 points
poly = PolynomialFeatures(degree = 2)
X_2 = poly.fit_transform(X)
Y_2 = Y

In [6]:
# CHECK - the shape of X_2 and Y_2 - should be (506, 105) and (506, 1) respectively
print(X_2.shape)
print(Y_2.shape)

(506, 105)
(506, 1)


Split both the sets of data into training and testing data to get a more accurate picture of real-world performance

In [7]:
# TODO - Call train_test_split for both X, Y and X_2, Y_2. Don't pass in any other parameters except for the dataset. 
# Let the split be the default value of 0.7 => 70% training and 30% test. - 10 points
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
X_2_train, X_2_test, Y_2_train, Y_2_test = train_test_split(X_2,Y_2)

# CHECK - Shapes of the resulting variables should be as follows:
# (379, 13) (127, 13) (379, 1) (127, 1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
# (379, 105) (127, 105) (379, 1) (127, 1)
print(X_2_train.shape, X_2_test.shape, Y_2_train.shape, Y_2_test.shape)

(354, 13) (152, 13) (354, 1) (152, 1)
(354, 105) (152, 105) (354, 1) (152, 1)


# Ridge Regression and Linear Regression

In [8]:
# TODO - Define the get_error_ridge function. Use Ridge() from sklearn.
# TODO - Return train_error and test_error values
# Some important functions to know are .fit() and .predict()
# HINT - fit() the data on training data and predict() on training and testing data to get the train and test error
# 15 points

def get_error_ridge(X_train, X_test, Y_train, Y_test, alpha):
    model = sklearn.linear_model.Ridge(alpha = alpha)
    fit = model.fit(X_train,Y_train)
    predictTrain = model.predict(X_train)
    predictTest = model.predict(X_test)
    
    train_error =np.sum(np.square(Y_train - predictTrain)) / (Y_train.shape[0]) 
    test_error = np.sum(np.square(Y_test - predictTest)) / (Y_test.shape[0])
    
    return train_error, test_error

In [9]:
# Call the function above with alpha = 0 - Notice that when alpha = 0, ridge regression and linear regression are equivalent
lin_rss = get_error_ridge(X_train, X_test, Y_train, Y_test, 0)
lin_poly_rss = get_error_ridge(X_2_train, X_2_test, Y_2_train, Y_2_test, 0)

print("For linear regression and no polynomial features: train_error = {0:.2f} and test_error = {1:.2f}".format(lin_rss[0], lin_rss[1]))
print("For linear regression and with polynomial features: train_error = {0:.2f} and test_error = {1:.2f}".format(lin_poly_rss[0], lin_poly_rss[1]))

For linear regression and no polynomial features: train_error = 18.46 and test_error = 31.45
For linear regression and with polynomial features: train_error = 5.68 and test_error = 16.42


In [10]:
# Call the function above with alpha = 1
lin_rss = get_error_ridge(X_train, X_test, Y_train, Y_test,1)
lin_poly_rss = get_error_ridge(X_2_train, X_2_test, Y_2_train, Y_2_test, 1 )

print("For ridge regression and no polynomial features: train_error = {0:.2f} and test_error = {1:.2f}".format(lin_rss[0], lin_rss[1]))
print("For ridge regression and with polynomial features: train_error = {0:.2f} and test_error = {1:.2f}".format(lin_poly_rss[0], lin_poly_rss[1]))

For ridge regression and no polynomial features: train_error = 18.46 and test_error = 31.44
For ridge regression and with polynomial features: train_error = 5.83 and test_error = 14.45


In [11]:
# TODO - Define the get_coeff_ridge_normaleq function. Use the normal equation method.
# TODO - Return w values
# 10 points

def get_coeff_ridge_normaleq(X_train, Y_train, alpha):
    # a = X.T.(X) + alpha * np.identity(# of columns in X)
    a = np.transpose(X_train).dot(X_train) + alpha * np.identity(X_train.shape[1])
    # q is the pseudo inverse of a
    q = np.linalg.pinv(a)
    # z is X.T.dot(Y)
    z = np.transpose(X_train).dot(Y_train)
    # w = q dot z - reshape to (# columns in X, 1)
    w = np.reshape(q.dot(z),(X_train.shape[1],1))
    
    return w

In [12]:
# TODO - Define the evaluate_err_ridge_normaleq function.
# TODO - Return the train_error and test_error values
# 10 points

def evaluate_err_ridge_normaleq(X_train, X_test, Y_train, Y_test, w): 
    yhat = X_train.dot(w)
    yhatTest = X_test.dot(w)
    
    train_error = np.sum(np.square(Y_train - yhat)) / (Y_train.shape[0]) 
    test_error = np.sum(np.square(Y_test - yhatTest)) / (Y_test.shape[0])
    
    return train_error, test_error

In [13]:
# Call the function above with alpha = 0 - Notice that when alpha = 0, ridge regression and linear regression are equivalent
w = get_coeff_ridge_normaleq(X_train, Y_train, 0)
lin_rss = evaluate_err_ridge_normaleq(X_train, X_test, Y_train, Y_test, w)

w_2 = get_coeff_ridge_normaleq(X_2_train, Y_2_train, 0)
lin_poly_rss = evaluate_err_ridge_normaleq(X_2_train, X_2_test, Y_2_train, Y_2_test, w_2)

print("For linear regression and no polynomial features: train_error = {0:.2f} and test_error = {1:.2f}".format(lin_rss[0], lin_rss[1]))
print("For linear regression and with polynomial features: train_error = {0:.2f} and test_error = {1:.2f}".format(lin_poly_rss[0], lin_poly_rss[1]))

For linear regression and no polynomial features: train_error = 512.28 and test_error = 588.89
For linear regression and with polynomial features: train_error = 5.67 and test_error = 16.40


In [14]:
# Call the function above with alpha = 1
w = get_coeff_ridge_normaleq(X_train, Y_train, 1)
lin_rss = evaluate_err_ridge_normaleq(X_train, X_test, Y_train, Y_test, w)

w_2 = get_coeff_ridge_normaleq(X_2_train, Y_2_train, 1)
lin_poly_rss = evaluate_err_ridge_normaleq(X_2_train, X_2_test, Y_2_train, Y_2_test, w_2)

print("For ridge regression and no polynomial features: train_error = {0:.2f} and test_error = {1:.2f}".format(lin_rss[0], lin_rss[1]))
print("For ridge regression and with polynomial features: train_error = {0:.2f} and test_error = {1:.2f}".format(lin_poly_rss[0], lin_poly_rss[1]))

For ridge regression and no polynomial features: train_error = 512.29 and test_error = 588.82
For ridge regression and with polynomial features: train_error = 5.94 and test_error = 14.27
